##  Data Analytics test assignment - SQL queries

Nijat Mirzabayov
26/04/2025



**A. Statistics by country. Calculate the following metrics for each country from database:**
- Total number of users from this country
- Amount of users who made at least one deposit
- Average amount of deposit for the country
- Average amount of withdrawal for the country
Provide the output sorted by number of users.

```sql
SELECT 
u.country_code AS CountryCode,
COUNT(DISTINCT u.user_id) AS TotalUsers,
COUNT(DISTINCT CASE WHEN b.operation_type = 'deposit' THEN u.user_id END) AS UsersWithDeposit,
ROUND(AVG(CASE WHEN b.operation_type = 'deposit' THEN b.operation_amount_usd END),2) AS AvgDeposit,
ROUND(AVG(CASE WHEN b.operation_type = 'withdrawal' THEN b.operation_amount_usd END),2) AS AvgWithdrawal
FROM (
SELECT *  -- Subquery is needed to fix duplicate "user_id" in "users" table problem
FROM users u1
WHERE  registration_time = (
SELECT MIN(u2.registration_time) -- Assumption is made that "user_id" with earlier registration date is a genuine one
FROM users u2
WHERE u2.user_id = u1.user_id)
) u
LEFT JOIN balance b ON u.user_id = b.user_id
GROUP BY u.country_code
ORDER BY COUNT(DISTINCT u.user_id) DESC;
```


**B. Active user. Find a user with the higher amount of profit from trading activity and
calculate some metrics for him. The expected output format is as follows:**
- ID of this user
- His country code
- His profit
- Total amount of deals
- Amount of profitable deals
- The most popular trading instrument (symbol). The position with the highest
amount of opened orders for this user
- The symbol with the highest level of profit
- The symbol with the highest level of loss

```sql
SELECT 
 u.user_id AS User,
 u.country_code AS CountryCOde,
 SUM(o.profit_usd) AS ProfitUSD,
 COUNT(*) AS NumberOfDeals,
 SUM(CASE WHEN o.profit_usd > 0 THEN 1 ELSE 0 END) AS NumberOfProfitableDeals,
--select subquery to get MostUsed symbol
(SELECT o1.symbol 
FROM orders o1
WHERE o1.user_id = o.user_id
GROUP BY o1.symbol
ORDER BY COUNT(*) DESC
LIMIT 1) AS MostUsedSymbol,
--select subquery to get HighestProfitSymbol
(SELECT o2.symbol
FROM orders o2
WHERE o2.user_id = o.user_id
GROUP BY o2.symbol
ORDER BY SUM(o2.profit_usd) DESC
LIMIT 1) AS HighestProfitSymbol,
--select subquery to get HighestProfitSymbol
(SELECT o3.symbol
FROM orders o3
WHERE o3.user_id = o.user_id
GROUP BY o3.symbol
ORDER BY SUM(o3.profit_usd) ASC
LIMIT 1) AS HighestLossSymbol

FROM 

(SELECT *  -- Subquery is needed to fix duplicate user_id in users table problem which causes incorrect aggregation when joining
FROM users u1
WHERE  registration_time = (
SELECT MIN(u2.registration_time) -- Assumption is made that user_id with earlier registration date is genuine one
FROM users u2
WHERE u2.user_id = u1.user_id)
) u
JOIN orders o 
WHERE u.user_id = o.user_id
GROUP BY u.user_id, u.country_code
ORDER BY SUM(o.profit_usd) DESC
LIMIT 1;
```

C. User’s funnel. Calculate the following metrics for each user
- User ID
- Country
- Registration datetime
- Date and time of the first deposit
- Date and time of the first trade (if any)
- Amount of the first deposit
- Profit / loss of the first trade
- Total deposit for the first 30 days since registration
- Total withdrawal for the first 30 days after registration
- Total profit / loss for the first 30 days after registration
- TOTAL profit / loss for the user’s lifetime

```sql
SELECT
u.user_id AS User,
u.country_code AS CountryCode,
u.registration_time AS RegistrationDate,

--Date and time of the first deposit
(SELECT MIN(b.operation_time)
FROM balance b
WHERE b.user_id = u.user_id AND b.operation_type = 'deposit') AS TimeOfFirstDeposit,

--Date and time of the first trade
(SELECT MIN(o.open_time)
FROM orders o
WHERE o.user_id = u.user_id) AS TimeOfFirstTrade,

--Amount of the first deposit
(SELECT b2.operation_amount_usd
FROM balance b2
WHERE b2.user_id = u.user_id AND b2.operation_type = 'deposit'
ORDER BY b2.operation_time ASC
LIMIT 1) AS AmountOfFirstDeposit,

--Profit / loss of the first trade
(SELECT o2.profit_usd
FROM orders o2
WHERE o2.user_id = u.user_id
ORDER BY o2.open_time ASC
LIMIT 1) AS ProfitLossOfFirstTrade,

--Total deposit for the first 30 days since registration
(SELECT SUM(b3.operation_amount_usd)
FROM balance b3
WHERE b3.user_id = u.user_id AND b3.operation_type = 'deposit'
AND b3.operation_time >= u.registration_time AND b3.operation_time <= datetime(u.registration_time, '+30 days')
) AS DepositFirst30days,

--Total withdrawal for the first 30 days after registration
(SELECT SUM(b4.operation_amount_usd)
FROM balance b4
WHERE b4.user_id = u.user_id AND b4.operation_type = 'withdrawal'
AND b4.operation_time >= u.registration_time
AND b4.operation_time <= datetime(u.registration_time, '+30 days')
) AS WithdrawalFirst30days,

-- Total profit / loss for the first 30 days after registration
(SELECT SUM(o3.profit_usd)
FROM orders o3
WHERE o3.user_id = u.user_id AND o3.open_time >= u.registration_time
AND o3.open_time <= datetime(u.registration_time, '+30 days')
) AS ProfitLoss30days,

--TOTAL profit / loss for the user’s lifetime
(SELECT SUM(o4.profit_usd)
FROM orders o4
WHERE o4.user_id = u.user_id
) AS TotalProfitLoss

FROM (SELECT *  -- Subquery is needed to fix duplicate user_id in users table problem which causes incorrect aggregation when joining
FROM users u1
WHERE  registration_time = (
SELECT MIN(u2.registration_time) -- Assumption is made that user_id with earlier registration date is genuine one
FROM users u2
WHERE u2.user_id = u1.user_id)
) u
```